In [1]:
! pip install transformers

%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

     |████████████████████████████████| 1.1MB 4.7MB/s 
     |████████████████████████████████| 3.0MB 24.1MB/s 
     |████████████████████████████████| 1.1MB 52.3MB/s 
     |████████████████████████████████| 890kB 54.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=943efdbd97f47b1082b9888a1f2542d4aec24d062d8b0b5c299571f13a95c259
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
2.3.0


## Fine-tuning BERT
- BERT Pre-Training phase: Masked Language Model(MLM) and Next Sentence Prediction(NSP) 

- Then, finetune all of the weights in the pretrained model for a task of choice.

- Source: https://arxiv.org/pdf/1810.04805.pdf

<img src="https://i.imgur.com/fKNAhKH.png" alt=" " width="75%" height="75%">



##  QuestionAnswering Model (inbuilt)
- Pretraining(NSP, MLM) + SQuAD Dataset trained model
- Extractive Model
- SQuAD: https://rajpurkar.github.io/SQuAD-explorer/

- "SQuAD2.0 combines the 100,000 questions in SQuAD1.1 with over 50,000 unanswerable questions written adversarially by crowdworkers to look similar to answerable ones. To do well on SQuAD2.0, systems must not only answer questions when possible, but also determine when no answer is supported by the paragraph and abstain from answering."



In [2]:
# Using HuggingFace's QuestionAnsweringModel (inbuilt)

import tensorflow as tf

# Refer: https://huggingface.co/transformers/model_doc/bert.html

from transformers import BertTokenizer, TFBertForQuestionAnswering 

modelName = 'bert-large-uncased-whole-word-masking-finetuned-squad' # https://huggingface.co/transformers/pretrained_models.html
 
tokenizer = BertTokenizer.from_pretrained(modelName)
model = TFBertForQuestionAnswering.from_pretrained(modelName)

print(model)

All model checkpoint weights were used when initializing TFBertForQuestionAnswering.

All the weights of TFBertForQuestionAnswering were initialized from the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


In [12]:
# Text+Q&A from SQuAD Test data: https://rajpurkar.github.io/SQuAD-explorer/explore/v2.0/dev/Amazon_rainforest.html?model=nlnet%20(single%20model)%20(Microsoft%20Research%20Asia)&version=v2.0
text = r"""The Amazon rainforest (Portuguese: Floresta Amazônica or Amazônia; Spanish: Selva Amazónica, Amazonía or usually Amazonia; French: Forêt amazonienne; Dutch: Amazoneregenwoud), also known in English as Amazonia or the Amazon Jungle, is a moist broadleaf forest that covers most of the Amazon basin of South America. This basin encompasses 7,000,000 square kilometres (2,700,000 sq mi), of which 5,500,000 square kilometres (2,100,000 sq mi) are covered by the rainforest. This region includes territory belonging to nine nations. The majority of the forest is contained within Brazil, with 60% of the rainforest, followed by Peru with 13%, Colombia with 10%, and with minor amounts in Venezuela, Ecuador, Bolivia, Guyana, Suriname and French Guiana. States or departments in four nations contain "Amazonas" in their names. The Amazon represents over half of the planet's remaining rainforests, and comprises the largest and most biodiverse tract of tropical rainforest in the world, with an estimated 390 billion individual trees divided into 16,000 species."""
#question =r"""What percentage does the Amazon represents in rainforests on the planet?"""

question = r"""How many nations contain "Amazonas" in their names?"""#


#input_text = "[CLS] " + question + " [SEP] " + text + " [SEP]"
input_text =  question + " [SEP] " + text 
input_ids = tokenizer.encode(input_text)

print(len(input_ids))
print(tokenizer.decode(input_ids))

input = tf.constant(input_ids)[None, :]  # Batch size 1 


249
[CLS] how many nations contain " amazonas " in their names? [SEP] the amazon rainforest ( portuguese : floresta amazonica or amazonia ; spanish : selva amazonica, amazonia or usually amazonia ; french : foret amazonienne ; dutch : amazoneregenwoud ), also known in english as amazonia or the amazon jungle, is a moist broadleaf forest that covers most of the amazon basin of south america. this basin encompasses 7, 000, 000 square kilometres ( 2, 700, 000 sq mi ), of which 5, 500, 000 square kilometres ( 2, 100, 000 sq mi ) are covered by the rainforest. this region includes territory belonging to nine nations. the majority of the forest is contained within brazil, with 60 % of the rainforest, followed by peru with 13 %, colombia with 10 %, and with minor amounts in venezuela, ecuador, bolivia, guyana, suriname and french guiana. states or departments in four nations contain " amazonas " in their names. the amazon represents over half of the planet's remaining rainforests, and compris

In [13]:
# Source: Modified PyTorch code from https://www.kaggle.com/c/tensorflow2-question-answering/discussion/123434

#[SEP] =  102 (token_id)
# token_type_ids has 0 corresponding to the qeustion part of the input_text and 1 corresponding to the 
# text/part 

token_type_ids = [0 if i <= input_ids.index(102) else 1 for i in range(len(input_ids))]

print(token_type_ids)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [14]:
answer=model(input, token_type_ids = tf.convert_to_tensor([token_type_ids]))

print(type(answer))
print(len(answer))

# https://huggingface.co/transformers/model_doc/distilbert.html#tfdistilbertforquestionanswering

startScores, endScores = answer
print(startScores.shape)
print(endScores.shape)
print(type(startScores))




<class 'tuple'>
2
(1, 249)
(1, 249)
<class 'tensorflow.python.framework.ops.EagerTensor'>


In [15]:
input_tokens = tokenizer.convert_ids_to_tokens(input_ids)

print(input_tokens)

['[CLS]', 'how', 'many', 'nations', 'contain', '"', 'amazon', '##as', '"', 'in', 'their', 'names', '?', '[SEP]', 'the', 'amazon', 'rainforest', '(', 'portuguese', ':', 'flores', '##ta', 'amazon', '##ica', 'or', 'amazon', '##ia', ';', 'spanish', ':', 'se', '##lva', 'amazon', '##ica', ',', 'amazon', '##ia', 'or', 'usually', 'amazon', '##ia', ';', 'french', ':', 'fore', '##t', 'amazon', '##ien', '##ne', ';', 'dutch', ':', 'amazon', '##ere', '##gen', '##wo', '##ud', ')', ',', 'also', 'known', 'in', 'english', 'as', 'amazon', '##ia', 'or', 'the', 'amazon', 'jungle', ',', 'is', 'a', 'moist', 'broad', '##leaf', 'forest', 'that', 'covers', 'most', 'of', 'the', 'amazon', 'basin', 'of', 'south', 'america', '.', 'this', 'basin', 'encompasses', '7', ',', '000', ',', '000', 'square', 'kilometres', '(', '2', ',', '700', ',', '000', 'sq', 'mi', ')', ',', 'of', 'which', '5', ',', '500', ',', '000', 'square', 'kilometres', '(', '2', ',', '100', ',', '000', 'sq', 'mi', ')', 'are', 'covered', 'by', 'the'

In [16]:
print(startScores)


tf.Tensor(
[[-6.044892   -7.0973377  -8.197169   -8.944285   -8.671532   -8.266655
  -8.501142   -9.148466   -8.985579   -8.4178915  -8.38475    -9.434322
  -9.458631   -6.0448794  -6.639209   -5.9593997  -7.28779    -8.263239
  -6.77111    -8.486906   -7.3450136  -8.412874   -7.779695   -8.274897
  -8.744188   -7.105765   -8.311125   -8.51701    -6.7672505  -8.69853
  -7.87896    -8.472504   -8.090377   -8.559998   -9.030277   -7.7963967
  -8.733086   -8.756392   -7.3531165  -7.7456326  -8.586231   -8.519922
  -5.75431    -8.305259   -7.000528   -8.180002   -7.5251684  -8.585446
  -8.396916   -8.661643   -6.8455057  -8.47022    -7.2067037  -8.718186
  -8.61971    -8.638312   -8.333733   -7.4745345  -8.208648   -8.022166
  -8.079031   -8.312786   -7.2532587  -8.43431    -7.2637258  -8.537678
  -8.6612425  -7.6164436  -6.910886   -8.118418   -8.06657    -7.3597035
  -7.474151   -7.5842004  -8.026866   -8.552894   -7.358252   -8.159294
  -7.4815564  -7.3753057  -8.466668   -7.5811644  -6

In [17]:
print(tf.math.argmax(startScores[0],0)) # https://www.tensorflow.org/api_docs/python/tf/math/argmax
    

tf.Tensor(192, shape=(), dtype=int64)


In [18]:
print((tf.math.argmax(startScores[0],0)).numpy())

192


In [19]:
startIdx = tf.math.argmax(startScores[0],0).numpy()
endIdx = tf.math.argmax(endScores[0],0).numpy()+1
print(startIdx,endIdx)

192 193


In [20]:
print(" ".join(input_tokens[startIdx:endIdx]))

four


## Additional Resources
- Mobile BERT QA System (code in pure TF/Keras):  https://www.tensorflow.org/lite/models/bert_qa/overview

- https://towardsdatascience.com/testing-bert-based-question-answering-on-coronavirus-articles-13623637a4ff

- Finetune on custom-data: https://huggingface.co/transformers/examples.html#squad